In [1]:
import numpy as np
import torch
from ase.io import read, write
from ase import Atoms
from dpyscfl.utils import ase_atoms_to_mol, gen_mf_mol

# Case study: O3+O

In [2]:
# training dataset, O3 is trj[24], O is 20
trj = read('/home/awills/Documents/Research/sebshare/data/09/subat_ref.traj', ':')
g2o3 = trj[24]
print(g2o3.positions)
print(g2o3.info)
g2o = trj[20]
print(g2o.positions)
print(g2o.info)

[[ 0.        1.075722 -0.21567 ]
 [ 0.        0.        0.43134 ]
 [ 0.       -1.075722 -0.21567 ]]
{'O3': True, 'name': 'Ozone', 'openshell': False, 'sc': False, 'sym': False, 'n_rad': 1, 'grid_level': 1, 'pol': False, 'target_energy': -0.2345382154735096, 'energy': -225.20206708569773, 'atomization': -0.2345382154735096, 'atomization_ev': -6.382109918715892, 'atomization_H': -0.2345382154735096, 'calc_energy': -225.20206708569773, 'e_calc': -225.20206708569773}
[[0. 0. 0.]]
{'spin': 2, 'sym': False, 'grid_level': 4, 'basis': '6-311++G(3df,2pd)', 'pol': False, 'target_energy': -75.0673, 'energy': -74.99416120454062, 'atomization': -75.0673, 'atomization_ev': -2042.6852781069833, 'atomization_H': -75.0673, 'calc_energy': -75.0673, 'e_calc': -75.0673}


/home/awills/anaconda3/envs/oldpyscf/lib/python3.7/site-packages/ase/io/jsonio.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.array(obj)


In [3]:
for ind, at in enumerate(trj):
    print(ind, at)

0 Atoms(symbols='H2', pbc=False, calculator=SinglePointCalculator(...))
1 Atoms(symbols='N2', pbc=False, calculator=SinglePointCalculator(...))
2 Atoms(symbols='LiF', pbc=False, calculator=SinglePointCalculator(...))
3 Atoms(symbols='CNH', pbc=False, calculator=SinglePointCalculator(...))
4 Atoms(symbols='CO2', pbc=False, calculator=SinglePointCalculator(...))
5 Atoms(symbols='Cl2', pbc=False, calculator=SinglePointCalculator(...))
6 Atoms(symbols='F2', pbc=False, calculator=SinglePointCalculator(...))
7 Atoms(symbols='O2', pbc=False, calculator=SinglePointCalculator(...))
8 Atoms(symbols='C2H2', pbc=False, calculator=SinglePointCalculator(...))
9 Atoms(symbols='OC', pbc=False, calculator=SinglePointCalculator(...))
10 Atoms(symbols='ClH', pbc=False, calculator=SinglePointCalculator(...))
11 Atoms(symbols='LiH', pbc=False, calculator=SinglePointCalculator(...))
12 Atoms(symbols='Na2', pbc=False, calculator=SinglePointCalculator(...))
13 Atoms(symbols='C', pbc=False, calculator=SinglePo

In [4]:
#positions from training dataset, but recreation to show how one makes the Atoms object
o3 = Atoms(['O','O','O'], positions = [[ 0.,        1.075722, -0.21567 ], 
                                       [ 0.,        0.,        0.43134 ],
                                       [ 0.,       -1.075722, -0.21567 ]])
o3.info = {'O3': True, 
           'name': 'Ozone', 
           'openshell': False, 
           'sc': False, 
           'sym': False, 
           'n_rad': 1, 
           'grid_level': 1, 
           'pol': False, 
           'target_energy': -0.2345382154735096, 
           'energy': -225.20206708569773, 
           'atomization': -0.2345382154735096, 
           'atomization_ev': -6.382109918715892, 
           'atomization_H': -0.2345382154735096, 
           'calc_energy': -225.20206708569773, 
           'e_calc': -225.20206708569773}
o = Atoms(['O'], positions=[[0,0,0]])
o.info = {'spin': 2, 
          'sym': False, 
          'grid_level': 4, 
          'basis': '6-311++G(3df,2pd)', 
          'pol': False, 
          'target_energy': -75.0673, 
          'energy': -74.99416120454062, 
          'atomization': -75.0673, 
          'atomization_ev': -2042.6852781069833, 
          'atomization_H': -75.0673, 
          'calc_energy': -75.0673, 
          'e_calc': -75.0673}

#openshell is false, 0 spin ozone -- generate the pyscf Mol objects to be used below
name_o3, mol_o3 = ase_atoms_to_mol(o3, basis='def2-tzvp', charge=0, spin=0)
name_o, mol_o = ase_atoms_to_mol(o, basis='def2-tzvp', charge=0, spin=2)

# PySCF

In [5]:
import pyscf
from pyscf import dft
print(pyscf.__file__)
print(pyscf.__version__)

/home/awills/anaconda3/envs/oldpyscf/lib/python3.7/site-packages/pyscf/__init__.py
1.7.6


In [6]:
#generate the mf and method from the mol and options
xc = 'pbe'
mf1_o3, method1_o3 = gen_mf_mol(mol_o3, xc=xc, pol=True, grid_level=3, nxc = False)
#run, save energy
mf1_o3.kernel()
e1_o3 = mf1_o3.e_tot

mf1_o, method1_o = gen_mf_mol(mol_o, xc=xc, pol=True, grid_level=3, nxc = False)
mf1_o.kernel()
e1_o = mf1_o.e_tot

print('PySCF ({}) Evaluated on:\nO3 = {}\nO = {}'.format(xc.upper(), e1_o3, e1_o))

Building grids...
Grid level = 3
Grid coordinate shape: (42786, 3)
METHOD GENERATED: <function UKS at 0x7f5491a753b0>
converged SCF energy = -225.323132026831  <S^2> = 3.375078e-14  2S+1 = 1
Single Atom -- UKS
Building grids...
Grid level = 3
Grid coordinate shape: (14262, 3)
METHOD GENERATED: <function UKS at 0x7f5491a753b0>
converged SCF energy = -75.0096713754199  <S^2> = 2.0027833  2S+1 = 3.0018549
PySCF (PBE) Evaluated on:
O3 = -225.3231320268307
O = -75.00967137541993


# Pylibnxc

In [7]:
#for pylibnxc version
import pylibnxc
from pylibnxc.pyscf import RKS, UKS
print(pylibnxc.__file__)

/home/awills/anaconda3/envs/oldpyscf/lib/python3.7/site-packages/pylibnxc/__init__.py


In [8]:
#location of XCDiff, the network is stored within in file 'xc'
# modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA/xc'
modelpath = '/home/awills/Documents/Research/swxcd/scont/target_val/MODEL_GGA_e00/xc'
#libnxc looks in cwd/MGGA_XC_CUSTOM for the xc network
try:
    os.mkdir('MGGA_XC_CUSTOM')
except:
    pass

try:
    os.symlink(modelpath, 'MGGA_XC_CUSTOM/xc')
except:
    pass

#this really just generates the method to pass into KS, since the mf is created in KS
_, method2_o3 = gen_mf_mol(mol_o3, xc='notnull', pol=True, grid_level=3, nxc=True)
#use wrapper to sent model to pylibnxc, generate and save energy
mf2_o3 = UKS(mol_o3, nxc='MGGA_XC_CUSTOM', nxc_kind='grid')
mf2_o3.kernel()
e2_o3 = mf2_o3.e_tot

_, method2_o = gen_mf_mol(mol_o, xc='notnull', pol=True, grid_level=3, nxc=True)
mf2_o = UKS(mol_o, nxc='MGGA_XC_CUSTOM', nxc_kind='grid')
mf2_o.kernel()
e2_o = mf2_o.e_tot

print('Pylibnxc (Pre-trained PBE) Evaluated on:\nO3 = {}\nO = {}'.format(e2_o3, e2_o))

Building grids...
Grid level = 3
Grid coordinate shape: (42786, 3)
METHOD GENERATED: functools.partial(<function KS at 0x7f54907c4b00>, method=<function UKS at 0x7f5491a753b0>)
converged SCF energy = -225.170147070068  <S^2> = 9.7948316e-12  2S+1 = 1
Single Atom -- UKS
Building grids...
Grid level = 3
Grid coordinate shape: (14262, 3)
METHOD GENERATED: functools.partial(<function KS at 0x7f54907c4b00>, method=<function UKS at 0x7f5491a753b0>)
converged SCF energy = -74.9819643915302  <S^2> = 2.0042966  2S+1 = 3.002863
Pylibnxc (Pre-trained PBE) Evaluated on:
O3 = -225.17014707006797
O = -74.98196439153021


# Dpyscfl

**WARNING: On my machine at least, PyTorch *does not release the memory used during an SCF cycle.* Doing the above calculations with 25 SCF steps uses ~50 GB of RAM, which might not get released until restarting this notebook's kernel. User beware.**

In [9]:
#for dpyscfl
import dpyscfl
from dpyscfl.net import get_scf
from dpyscfl.utils import old_get_datapoint
print(dpyscfl.__file__)

/home/awills/anaconda3/envs/oldpyscf/lib/python3.7/site-packages/dpyscfl/__init__.py


In [10]:
232.974-107.208

125.76599999999999

In [11]:
#generate pytorch matrices based on mol and options
mats_o3 = old_get_datapoint(o3, basis='6-311++G(3df,2pd)', grid_level= 3,
                                xc='scan', zsym=o3.info.get('sym', False),
                                n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
                                init_guess=False, spin = 0,
                                pol=True, do_fcenter=True,
                                ref_path='/home/awills/Documents/Research/sebshare/data/09', ref_index= 24, ref_traj='subat_ref.traj',
                                ref_basis='6-311++G(3df,2pd)', dfit=False)
mats_o = old_get_datapoint(o, basis='6-311++G(3df,2pd)', grid_level= 3,
                                xc='scan', zsym=o.info.get('sym', False),
                                n_rad=o.info.get('n_rad',30), n_ang=o.info.get('n_ang',15),
                                init_guess=False, spin = 2,
                                pol=True, do_fcenter=True,
                                ref_path='/home/awills/Documents/Research/sebshare/data/09', ref_index= 20, ref_traj='subat_ref.traj',
                                ref_basis='6-311++G(3df,2pd)', dfit=False)
#unpack returned tuples
e_base_o3, eye_o3, mats_o3 = mats_o3
e_base_o, eye_o, mats_o = mats_o
#need to generate more matrices for network -- this is usually handled by the data reader class used in training
s_chol_o3 = np.linalg.inv(np.linalg.cholesky(mats_o3['s']))
s_chol_o = np.linalg.inv(np.linalg.cholesky(mats_o['s']))

mats_o3['s_chol'] = s_chol_o3
mats_o3['s_oh'] = s_chol_o3
mats_o3['s_inv_oh'] = s_chol_o3.T
mats_o['s_chol'] = s_chol_o
mats_o['s_oh'] = s_chol_o
mats_o['s_inv_oh'] = s_chol_o.T

#go through and make everything a list, since everything in the scripts assumes matrices stored as 
# key: [mat_idx0, mat_idx1, ...]
mats_o3_l = {k: [mats_o3[k]] for k in mats_o3.keys()}
mats_o_l = {k: [mats_o[k]] for k in mats_o.keys()}

#go through and additionally make everything a tensor, since forward pass expects things to be tensors
for k in mats_o_l.keys():
    print(k)
    o3k = mats_o3_l[k]
    ok = mats_o_l[k]
    if type(ok[0]) in [int, float]:
        print('integer/float from o3 dict')
        mats_o3_l[k] = [torch.Tensor(np.array(o3k))]
        mats_o_l[k] = [torch.Tensor(np.array(ok))]
    elif not ok[0].shape:
        mats_o3_l[k] = [torch.Tensor(o3k[0].reshape(1,))]
        mats_o_l[k] = [torch.Tensor(ok[0].reshape(1,))]
    else:
        mats_o3_l[k] = [torch.Tensor(o3k[0])]
        mats_o_l[k] = [torch.Tensor(ok[0])]

Atoms(symbols='O3', pbc=False)
6-311++G(3df,2pd)
No specifications in atom info to help, assuming no spin.
Building grids...
Grid level = 3
Grid coordinate shape: (42786, 3)
METHOD GENERATED: <function UKS at 0x7f5491a753b0>
converged SCF energy = -225.426594202773  <S^2> = 6.1508132e-11  2S+1 = 1


/home/awills/anaconda3/envs/oldpyscf/lib/python3.7/site-packages/pyscf/gto/moleintor.py:252: UserWarning: Function int4c1e_sph not found.  Set its comp to 1
  warnings.warn('Function %s not found.  Set its comp to 1' % intor_name)


DATAPOINT E_BASE:  -225.42659420277272
DATAPOINT MO_ENERGY:  [[-1.92571100e+01 -1.90762671e+01 -1.90762518e+01 -1.34036886e+00
  -1.07060292e+00 -7.53192353e-01 -5.75704506e-01 -5.67891687e-01
  -5.61150410e-01 -3.48594750e-01 -3.15168471e-01 -3.05517948e-01
  -2.25212138e-01  3.81674387e-02  9.40075120e-02  1.01851496e-01
   1.41693816e-01  1.50795435e-01  1.75940012e-01  2.18854400e-01
   2.19385694e-01  2.19898701e-01  2.34668605e-01  2.40682916e-01
   3.02788037e-01  3.27238741e-01  3.67685060e-01  6.13849116e-01
   6.99187710e-01  7.29281308e-01  8.00291718e-01  8.38083067e-01
   8.62844724e-01  8.75352963e-01  9.12076168e-01  9.18789589e-01
   9.20247624e-01  9.46069888e-01  9.59753268e-01  1.00309848e+00
   1.00708075e+00  1.01139283e+00  1.02698762e+00  1.06247400e+00
   1.12188642e+00  1.23539808e+00  1.39037006e+00  1.41856166e+00
   1.45392406e+00  1.53416391e+00  1.71562933e+00  1.79968041e+00
   2.20064072e+00  2.39232935e+00  2.83183254e+00  2.86748171e+00
   2.96687913e+

In [12]:
old_get_datapoint??

In [14]:
def freeze_net(nn):
    for i in nn.parameters():
        i.requires_grad = False

#location of model
# modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA/xc'
modelpath = '/home/awills/Documents/Research/swxcd/scont/target_val/MODEL_GGA_e00/xc'
ptloc = '/home/awills/Documents/Research/dpyscfl/models/pretrained/pbe'
#type MGGA specified, but additionally get_scf expects the full file, so add /xc to path
scf = get_scf('GGA', path=modelpath)
#do self-consistent
sc=True
#added recently -- prints brief info each pass
vvv = True
#generate results dictionary and then select energy only
freeze_net(scf)
res_o3 = scf([torch.Tensor(mats_o3_l['dm_init'][0])], mats_o3_l, sc=sc, verbose=vvv)
res_o3 = res_o3['E']

res_o = scf([torch.Tensor(mats_o_l['dm_init'][0])], mats_o_l, sc=sc, verbose=vvv)
res_o = res_o['E']

print('Dpyscf-lite (Pre-Trained PBE) Evaluated on:\nO3 = {}'.format(res_o3))
print('Dpyscf-lite (Pre-Trained PBE) Evaluated on:\nO = {}'.format(res_o))

FREEC False
SCF Loop Beginning: 25 Steps
Step 0
Density Matrix stats: 
Mean:  tensor(0.0008)
Min/Max:  tensor(-0.2003) tensor(0.4622)
Select Indices: dm.flatten()[[0, 5, 10, 100]] tensor([ 3.2079e-01, -7.2276e-17,  2.9288e-03, -7.0353e-19])
Noise generation to avoid potential degeneracies
0 Energy: tensor([-223.8845])
History: [tensor([-223.8845])]
Step 1
Density Matrix stats: 
Mean:  tensor(0.0008)
Min/Max:  tensor(-0.2413) tensor(0.5316)
Select Indices: dm.flatten()[[0, 5, 10, 100]] tensor([3.1898e-01, 1.7974e-10, 3.0432e-03, 4.5100e-10])
1 Energy: tensor([-223.8874])
History: [tensor([-223.8845]), tensor([-223.8874])]
Step 2
Density Matrix stats: 
Mean:  tensor(0.0008)
Min/Max:  tensor(-0.2489) tensor(0.5446)
Select Indices: dm.flatten()[[0, 5, 10, 100]] tensor([ 3.1866e-01, -1.4992e-11,  3.1148e-03, -3.8821e-11])
2 Energy: tensor([-223.8883])
History: [tensor([-223.8845]), tensor([-223.8874]), tensor([-223.8883])]
Step 3
Density Matrix stats: 
Mean:  tensor(0.0008)
Min/Max:  tensor

19 Energy: tensor([-223.8895])
History: [tensor([-223.8845]), tensor([-223.8874]), tensor([-223.8883]), tensor([-223.8887]), tensor([-223.8890]), tensor([-223.8892]), tensor([-223.8893]), tensor([-223.8894]), tensor([-223.8895]), tensor([-223.8895]), tensor([-223.8896]), tensor([-223.8896]), tensor([-223.8896]), tensor([-223.8896]), tensor([-223.8896]), tensor([-223.8896]), tensor([-223.8896]), tensor([-223.8896]), tensor([-223.8896]), tensor([-223.8895])]
Step 20
Density Matrix stats: 
Mean:  tensor(0.0008)
Min/Max:  tensor(-0.2609) tensor(0.5639)
Select Indices: dm.flatten()[[0, 5, 10, 100]] tensor([3.1819e-01, 5.5028e-10, 3.2379e-03, 3.3890e-10])
20 Energy: tensor([-223.8894])
History: [tensor([-223.8845]), tensor([-223.8874]), tensor([-223.8883]), tensor([-223.8887]), tensor([-223.8890]), tensor([-223.8892]), tensor([-223.8893]), tensor([-223.8894]), tensor([-223.8895]), tensor([-223.8895]), tensor([-223.8896]), tensor([-223.8896]), tensor([-223.8896]), tensor([-223.8896]), tensor(

11 Energy: tensor([-74.5308])
History: [tensor([-74.5285]), tensor([-74.5297]), tensor([-74.5301]), tensor([-74.5303]), tensor([-74.5305]), tensor([-74.5306]), tensor([-74.5306]), tensor([-74.5307]), tensor([-74.5307]), tensor([-74.5307]), tensor([-74.5308]), tensor([-74.5308])]
Step 12
Density Matrix stats: 
Mean:  tensor(0.0033)
Min/Max:  tensor(-0.1341) tensor(0.4274)
Select Indices: dm.flatten()[[0, 5, 10, 100]] tensor([ 3.1807e-01, -5.6367e-10, -1.3026e-09, -4.9416e-03])
12 Energy: tensor([-74.5308])
History: [tensor([-74.5285]), tensor([-74.5297]), tensor([-74.5301]), tensor([-74.5303]), tensor([-74.5305]), tensor([-74.5306]), tensor([-74.5306]), tensor([-74.5307]), tensor([-74.5307]), tensor([-74.5307]), tensor([-74.5308]), tensor([-74.5308]), tensor([-74.5308])]
Step 13
Density Matrix stats: 
Mean:  tensor(0.0033)
Min/Max:  tensor(-0.1341) tensor(0.4274)
Select Indices: dm.flatten()[[0, 5, 10, 100]] tensor([ 3.1807e-01, -2.3304e-10, -1.8337e-09, -4.9397e-03])
13 Energy: tensor(

The output of the above cell is pasted at the end of the cell in the event the generated output above gets erased.

In [24]:
print(mats_o_l.keys())
print(mats2_o_l.keys())
for k in mats_o_l.keys():
    print(k)
    try:
        s1 = mats_o_l[k][0]
    except:
        print("Key does not exist in matrix 1")
        print('===================================================')
        continue
    try:
        s2 = mats2_o_l[k][0]
    except:
        print("Key does not exist in matrix 2")
        print('===================================================')
        continue
    mx1, mn1, sm1 = (torch.max(s1), torch.min(s1), torch.sum(s1))
    mx2, mn2, sm2 = (torch.max(s2), torch.min(s2), torch.sum(s2))
    
    p = torch.mean(abs(s1-s2))
    p2 = torch.max(abs(s1-s2))
    p3 = torch.min(abs(s1-s2))
    p4 = torch.sum(abs(s1-s2))
    print("Max/Min/Sum of matrix 1: {}/{}/{}".format(mx1,mn1,sm1))
    print("Max/Min/Sum of matrix 2: {}/{}/{}".format(mx2,mn2,sm2))
    print("MAE between matrices: {}".format(p))
    print("Max/Min absolute deviation between matrices: {}/{}".format(p2,p3))
    print("Sum of all absolute deviations: {}".format(p4))
    print('===================================================')

dict_keys(['dm_init', 'dm', 'v', 't', 's', 'eri', 'n_elec', 'n_atoms', 'e_nuc', 'mo_energy', 'mo_occ', 'mo_coeff', 'e_ip', 'ip_idx', 'e_pretrained', 'e_base', 'dm_realinit', 'f_center', 'L', 'scaling', 'ao_eval', 'grid_weights', 'e_calc', 'rho', 's_chol', 's_oh', 's_inv_oh'])
dict_keys(['dm_init', 'v', 't', 's', 'eri', 'n_elec', 'n_atoms', 'e_nuc', 'mo_energy', 'mo_occ', 'mo_coeff', 'e_ip', 'ip_idx', 'e_pretrained', 'dm_realinit', 'e_base', 'e_ref', 'L', 'scaling', 'rho', 'ao_eval', 'grid_weights', 'fcenter', 's_chol', 's_oh', 's_inv_oh'])
dm_init
Max/Min/Sum of matrix 1: 0.4368487440728951/-0.13608352856447797/10.098158850656741
Max/Min/Sum of matrix 2: 0.43696545609361676/-0.13611061795366217/10.852930175918397
MAE between matrices: 0.0023670856952028993
Max/Min absolute deviation between matrices: 0.14311498045488247/1.0406559426877107e-21
Sum of all absolute deviations: 7.200674684807219
dm
Key does not exist in matrix 2
v
Max/Min/Sum of matrix 1: 4.440892098500626e-16/-78.83309615

# Dpyscf (original, contained in xcdiff folder)

**WARNING: On my machine at least, PyTorch *does not release the memory used during an SCF cycle.* Doing the above calculations with 25 SCF steps uses ~50 GB of RAM, which might not get released until restarting this notebook's kernel. User beware.**

In [7]:
import dpyscf
from dpyscf.utils import get_datapoint
from dpyscf.net import lite_get_scf
print(dpyscf.__file__)

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/dpyscf/__init__.py


In [21]:
ref_path = '/home/awills/Documents/Research/swxcd/aegis/test_subset_ps2/09'
#data preparation is different
#if using old scripts from xcdiff/original dpyscf,
#reference density matrices must be in ref_path/{INDEX}.dm.npy
#can just symlink the DMs produced by run_pyscf.py
#ADDITIONALLY, reference energies must be stored in ref_path/energies
#formatted as:
#IDX ENERGY
# mats2_o3 = get_datapoint(o3, basis='6-311++G(3df,2pd)', grid_level= 3,
#                                 xc='scan', zsym=o3.info.get('sym', False),
#                                 n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
#                                 init_guess=False, spin = 0,
#                                 pol=True, do_fcenter=True,
#                                 ref_path=ref_path, ref_index= 24,
#                                 ref_basis='6-311++G(3df,2pd)')
mats2_o = get_datapoint(o, basis='6-311++G(3df,2pd)', grid_level= 3,
                                xc='scan', zsym=o.info.get('sym', False),
                                n_rad=o.info.get('n_rad',30), n_ang=o.info.get('n_ang',15),
                                init_guess=False, spin = o.info.get('spin', 0),
                                pol=True, do_fcenter=True,
                                ref_path=ref_path, ref_index= 20,
                                ref_basis='6-311++G(3df,2pd)',
                       grid_deriv=2)
#unpack returned tuples
# e_base2_o3, eye2_o3, mats2_o3 = mats2_o3
e_base2_o, eye2_o, mats2_o = mats2_o
#need to generate more matrices for network -- this is usually handled by the data reader class used in training
# s_chol_o3 = np.linalg.inv(np.linalg.cholesky(mats2_o3['s']))
s_chol_o = np.linalg.inv(np.linalg.cholesky(mats2_o['s']))

#old version expects this under different key names
# mats2_o3['s_chol'] = s_chol_o3
# mats2_o3['s_oh'] = s_chol_o3
# mats2_o3['s_inv_oh'] = s_chol_o3.T

mats2_o['s_chol'] = s_chol_o
mats2_o['s_oh'] = s_chol_o
mats2_o['s_inv_oh'] = s_chol_o.T

#go through and make everything a list
# mats2_o3_l = {k: [mats2_o3[k]] for k in mats2_o3.keys()}
mats2_o_l = {k: [mats2_o[k]] for k in mats2_o.keys()}

#go through and additionally make everything a tensor
for k in mats2_o_l.keys():
    print(k)
#     o3k = mats2_o3_l[k]
    ok = mats2_o_l[k]
    if type(ok[0]) in [int, float]:
        print('integer/float from o3 dict')
#         mats2_o3_l[k] = [torch.Tensor(np.array(o3k))]
        mats2_o_l[k] = [torch.Tensor(np.array(ok))]
    elif not ok[0].shape:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0].reshape(1,))]
        mats2_o_l[k] = [torch.Tensor(ok[0].reshape(1,))]
    else:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0])]
        mats2_o_l[k] = [torch.Tensor(ok[0])]

Atoms(symbols='O', pbc=False)
6-311++G(3df,2pd)
Spin specified in atom info.
Single Atom -- UKS
Building grids...
Grid level = 3
Grid coordinate shape: (14262, 3)
METHOD GENERATED: <function UKS at 0x7f6be5190320>
converged SCF energy = -75.0647816832198  <S^2> = 2.0054729  2S+1 = 3.0036464
DATAPOINT E_BASE:  -75.06478168321979
DATAPOINT MO_ENERGY:  [[-19.08776921  -0.9908762   -0.42755626  -0.42754348  -0.34414035
    0.10412159   0.16776598   0.16776837   0.17614622   0.73407755
    0.73408214   0.75851068   0.75852429   0.76867365   0.94728386
    0.94729748   0.99659362   1.15968113   3.21907898   3.21908595
    3.28041335   3.28043222   3.30527047   4.81804268   4.81806384
    4.89002773   4.95889541   4.9588984    5.01248797   5.01249018
    5.04806898   5.04809156   5.06438274  14.21097118  14.21097899
   14.27521012  14.27526169  14.30323945  49.65928148]
 [-19.02738324  -0.80390949  -0.29234028  -0.20654125  -0.20623593
    0.14269453   0.22227339   0.22238793   0.23136489   0

In [55]:
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA'
scf2 = lite_get_scf('MGGA', path=modelpath+'/xc')
# scf2 = scf2.float()

sc=True
vvv=True

# res2_o3 = scf2([torch.Tensor(mats2_o3_l['dm_init'][0])], mats2_o3_l, sc=sc, verbose=vvv)
# res2_o3 = res2_o3['E']
dminit = torch.Tensor(mats_o_l['dm_realinit'][0])
res2_o = scf2([dminit], mats_o_l, sc=sc, verbose=vvv)
res2_o = res2_o['E']

# print('Dpyscf (XCDiff) Evaluated on:\nO3 = {}'.format(res2_o3))
print('Dpyscf (XCDiff) Evaluated on:\nO = {}'.format(res2_o))

FREEC False
SCF Loop Beginning: 25 Steps
Step 0
Linear mixing for DM.
Density Matrix stats: 
Mean:  tensor(0.0031)
Min/Max:  tensor(-0.1348) tensor(0.4109)
Select Indices: dm.flatten()[[0, 5, 10, 100]] tensor([0.3237, 0.0000, 0.0000, 0.0000])
0 Energy: tensor([-74.8715], grad_fn=<AddBackward0>)
History: [tensor([-74.8715], grad_fn=<AddBackward0>)]
Step 1
Linear mixing for DM.
Density Matrix stats: 
Mean:  tensor(0.0022, grad_fn=<MeanBackward0>)
Min/Max:  tensor(-0.8634, grad_fn=<MinBackward1>) tensor(2.0028, grad_fn=<MaxBackward1>)
Select Indices: dm.flatten()[[0, 5, 10, 100]] tensor([ 6.5200e-01,  2.7946e-07, -1.2001e-06, -2.4641e-06],
       grad_fn=<IndexBackward0>)
1 Energy: tensor([-43.0654], grad_fn=<AddBackward0>)
History: [tensor([-74.8715], grad_fn=<AddBackward0>), tensor([-43.0654], grad_fn=<AddBackward0>)]
Step 2
Linear mixing for DM.
Density Matrix stats: 
Mean:  tensor(0.0021, grad_fn=<MeanBackward0>)
Min/Max:  tensor(-1.1889, grad_fn=<MinBackward1>) tensor(2.4799, grad_fn

12 Energy: tensor([-18.7596], grad_fn=<AddBackward0>)
History: [tensor([-74.8715], grad_fn=<AddBackward0>), tensor([-43.0654], grad_fn=<AddBackward0>), tensor([-34.1727], grad_fn=<AddBackward0>), tensor([-29.3857], grad_fn=<AddBackward0>), tensor([-26.2549], grad_fn=<AddBackward0>), tensor([-24.0551], grad_fn=<AddBackward0>), tensor([-22.4180], grad_fn=<AddBackward0>), tensor([-21.2812], grad_fn=<AddBackward0>), tensor([-20.4532], grad_fn=<AddBackward0>), tensor([-19.8266], grad_fn=<AddBackward0>), tensor([-19.3454], grad_fn=<AddBackward0>), tensor([-19.0224], grad_fn=<AddBackward0>), tensor([-18.7596], grad_fn=<AddBackward0>)]
Step 13
Linear mixing for DM.
Density Matrix stats: 
Mean:  tensor(0.0021, grad_fn=<MeanBackward0>)
Min/Max:  tensor(-1.7714, grad_fn=<MinBackward1>) tensor(3.2791, grad_fn=<MaxBackward1>)
Select Indices: dm.flatten()[[0, 5, 10, 100]] tensor([9.7128e-01, 1.4327e-06, 5.9143e-06, 6.7220e-02],
       grad_fn=<IndexBackward0>)
13 Energy: tensor([-18.5717], grad_fn=<

20 Energy: tensor([-17.9919], grad_fn=<AddBackward0>)
History: [tensor([-74.8715], grad_fn=<AddBackward0>), tensor([-43.0654], grad_fn=<AddBackward0>), tensor([-34.1727], grad_fn=<AddBackward0>), tensor([-29.3857], grad_fn=<AddBackward0>), tensor([-26.2549], grad_fn=<AddBackward0>), tensor([-24.0551], grad_fn=<AddBackward0>), tensor([-22.4180], grad_fn=<AddBackward0>), tensor([-21.2812], grad_fn=<AddBackward0>), tensor([-20.4532], grad_fn=<AddBackward0>), tensor([-19.8266], grad_fn=<AddBackward0>), tensor([-19.3454], grad_fn=<AddBackward0>), tensor([-19.0224], grad_fn=<AddBackward0>), tensor([-18.7596], grad_fn=<AddBackward0>), tensor([-18.5717], grad_fn=<AddBackward0>), tensor([-18.4263], grad_fn=<AddBackward0>), tensor([-18.3280], grad_fn=<AddBackward0>), tensor([-18.2461], grad_fn=<AddBackward0>), tensor([-18.1638], grad_fn=<AddBackward0>), tensor([-18.0777], grad_fn=<AddBackward0>), tensor([-18.0180], grad_fn=<AddBackward0>), tensor([-17.9919], grad_fn=<AddBackward0>)]
Step 21
Line

# Dpyscf (original, contained in standalone repo folder, hosted [here](https://github.com/semodi/dpyscf))

**WARNING: On my machine at least, PyTorch *does not release the memory used during an SCF cycle.* Doing the above calculations with 25 SCF steps uses ~50 GB of RAM, which might not get released until restarting this notebook's kernel. User beware.**

In [4]:
#for original+xcdiff dpyscf
import ogdpyscf
from ogdpyscf.utils import get_datapoint as og_get_datapoint
from ogdpyscf.net import lite_get_scf
print(ogdpyscf.__file__)

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/ogdpyscf/__init__.py


In [5]:
ref_path = '/home/awills/Documents/Research/swxcd/aegis/test_subset_ps2/09'
#data preparation is different
#if using old scripts from xcdiff/original dpyscf,
#reference density matrices must be in ref_path/{INDEX}.dm.npy
#can just symlink the DMs produced by run_pyscf.py
#ADDITIONALLY, reference energies must be stored in ref_path/energies
#formatted as:
#IDX ENERGY
# mats2_o3 = get_datapoint(o3, basis='6-311++G(3df,2pd)', grid_level= 3,
#                                 xc='scan', zsym=o3.info.get('sym', False),
#                                 n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
#                                 init_guess=False, spin = 0,
#                                 pol=True, do_fcenter=True,
#                                 ref_path=ref_path, ref_index= 24,
#                                 ref_basis='6-311++G(3df,2pd)')
mats2_o = og_get_datapoint(o, basis='6-311++G(3df,2pd)', grid_level= 5,
                                xc='scan', zsym=o.info.get('sym', True),
                                n_rad=o.info.get('n_rad',30), n_ang=o.info.get('n_ang',15),
                                init_guess=True, spin = o.info.get('spin', 0),
                                pol=o.info.get('pol', True), do_fcenter=True,
                                ref_path=ref_path, ref_index= 20,
                                ref_basis='6-311++G(3df,2pd)')
#unpack returned tuples
# e_base2_o3, eye2_o3, mats2_o3 = mats2_o3
e_base2_o, eye2_o, mats2_o = mats2_o

Atoms(symbols='O', pbc=False)
6-311++G(3df,2pd)


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/pyscf/gto/moleintor.py:259: UserWarning: Function int4c1e_sph not found.  Set its comp to 1
  warnings.warn('Function %s not found.  Set its comp to 1' % intor_name)


converged SCF energy = -75.0641912132701  <S^2> = 2.0054439  2S+1 = 3.0036271
Loading reference density
converged SCF energy = -75.064199835836  <S^2> = 2.00545  2S+1 = 3.0036311


In [6]:
#need to generate more matrices for network -- this is usually handled by the data reader class used in training
# s_chol_o3 = np.linalg.inv(np.linalg.cholesky(mats2_o3['s']))
s_chol_o = np.linalg.inv(np.linalg.cholesky(mats2_o['s']))

#old version expects this under different key names
# mats2_o3['s_chol'] = s_chol_o3
# mats2_o3['s_oh'] = s_chol_o3
# mats2_o3['s_inv_oh'] = s_chol_o3.T

mats2_o['s_chol'] = s_chol_o
mats2_o['s_oh'] = s_chol_o
mats2_o['s_inv_oh'] = s_chol_o.T

#go through and make everything a list
# mats2_o3_l = {k: [mats2_o3[k]] for k in mats2_o3.keys()}
mats2_o_l = {k: [mats2_o[k]] for k in mats2_o.keys()}

#go through and additionally make everything a tensor
for k in mats2_o_l.keys():
    print(k)
#     o3k = mats2_o3_l[k]
    ok = mats2_o_l[k]
    if type(ok[0]) in [int, float]:
        print('integer/float from o3 dict')
#         mats2_o3_l[k] = [torch.Tensor(np.array(o3k))]
        mats2_o_l[k] = [torch.Tensor(np.array(ok))]
    elif not ok[0].shape:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0].reshape(1,))]
        mats2_o_l[k] = [torch.Tensor(ok[0].reshape(1,))]
    else:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0])]
        mats2_o_l[k] = [torch.Tensor(ok[0])]

dm_init
v
t
s
n_elec
n_atoms
e_nuc
mo_energy
mo_occ
e_ip
ip_idx
e_pretrained
eri
L
scaling
rho
ao_eval
grid_weights
fcenter
s_chol
s_oh
s_inv_oh


In [7]:
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA'
scf3 = lite_get_scf('MGGA', path=modelpath+'/xc')

sc=True
vvv=True

# res2_o3 = scf3([torch.Tensor(mats2_o3_l['dm_init'][0])], mats2_o3_l, sc=sc, verbose=vvv)
# res2_o3 = res2_o3['E']

res2_o = scf3([torch.Tensor(mats2_o_l['dm_init'][0])], mats2_o_l, sc=sc, verbose=vvv)
res2_o = res2_o['E']

# print('Dpyscf (Standalone Repo) Evaluated on:\nO3 = {}'.format(res2_o3))
print('Dpyscf (Standalone Repo) Evaluated on:\nO = {}'.format(res2_o))

FREEC False
SCF Loop Beginning: 25 Steps
Step 0


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)
/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/ogdpyscf/torch_routines.py:77: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2524.)
  e, c = torch.symeig(co

0 Energy: tensor([-74.8328], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-74.8646], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-74.8862], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-74.9006], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>), tensor([-74.9006], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-74.9110], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>), tensor([-74.9006], grad_fn=<AddBackward0>), tensor([-74.9110], grad_fn=<AddBackward0>

18 Energy: tensor([-74.9383], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>), tensor([-74.9006], grad_fn=<AddBackward0>), tensor([-74.9110], grad_fn=<AddBackward0>), tensor([-74.9187], grad_fn=<AddBackward0>), tensor([-74.9244], grad_fn=<AddBackward0>), tensor([-74.9284], grad_fn=<AddBackward0>), tensor([-74.9314], grad_fn=<AddBackward0>), tensor([-74.9335], grad_fn=<AddBackward0>), tensor([-74.9349], grad_fn=<AddBackward0>), tensor([-74.9360], grad_fn=<AddBackward0>), tensor([-74.9367], grad_fn=<AddBackward0>), tensor([-74.9373], grad_fn=<AddBackward0>), tensor([-74.9376], grad_fn=<AddBackward0>), tensor([-74.9379], grad_fn=<AddBackward0>), tensor([-74.9381], grad_fn=<AddBackward0>), tensor([-74.9382], grad_fn=<AddBackward0>), tensor([-74.9383], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-74.9384], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<

# Dpyscf (original, contained in standalone repo folder, hosted [here](https://github.com/semodi/dpyscf), BUT includes Softplus in C_L network)

**WARNING: On my machine at least, PyTorch *does not release the memory used during an SCF cycle.* Doing the above calculations with 25 SCF steps uses ~50 GB of RAM, which might not get released until restarting this notebook's kernel. User beware.**

In [3]:
#for original+xcdiff dpyscf
import ogSdpyscf
from ogSdpyscf.utils import get_datapoint as og_get_datapoint
from ogSdpyscf.net import lite_get_scf
print(ogSdpyscf.__file__)

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/ogSdpyscf/__init__.py


In [8]:
ref_path = '/home/awills/Documents/Research/swxcd/aegis/test_subset_ps2/09'
#data preparation is different
#if using old scripts from xcdiff/original dpyscf,
#reference density matrices must be in ref_path/{INDEX}.dm.npy
#can just symlink the DMs produced by run_pyscf.py
#ADDITIONALLY, reference energies must be stored in ref_path/energies
#formatted as:
#IDX ENERGY
# mats2_o3 = get_datapoint(o3, basis='6-311++G(3df,2pd)', grid_level= 3,
#                                 xc='scan', zsym=o3.info.get('sym', False),
#                                 n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
#                                 init_guess=False, spin = 0,
#                                 pol=True, do_fcenter=True,
#                                 ref_path=ref_path, ref_index= 24,
#                                 ref_basis='6-311++G(3df,2pd)')
mats2_o = og_get_datapoint(o, basis='6-311++G(3df,2pd)', grid_level= 5,
                                xc='scan', zsym=o.info.get('sym', True),
                                n_rad=o.info.get('n_rad',30), n_ang=o.info.get('n_ang',15),
                                init_guess=True, spin = o.info.get('spin', 0),
                                pol=o.info.get('pol', True), do_fcenter=True,
                                ref_path=ref_path, ref_index= 20,
                                ref_basis='6-311++G(3df,2pd)')
#unpack returned tuples
# e_base2_o3, eye2_o3, mats2_o3 = mats2_o3
e_base2_o, eye2_o, mats2_o = mats2_o

Atoms(symbols='O', pbc=False)
6-311++G(3df,2pd)


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/pyscf/gto/moleintor.py:259: UserWarning: Function int4c1e_sph not found.  Set its comp to 1
  warnings.warn('Function %s not found.  Set its comp to 1' % intor_name)


converged SCF energy = -75.0641672113342  <S^2> = 2.005458  2S+1 = 3.0036365
Loading reference density
SCF not converged.
SCF energy = -75.0641792248053 after 50 cycles  <S^2> = 2.0054126  2S+1 = 3.0036062


In [9]:
#need to generate more matrices for network -- this is usually handled by the data reader class used in training
# s_chol_o3 = np.linalg.inv(np.linalg.cholesky(mats2_o3['s']))
s_chol_o = np.linalg.inv(np.linalg.cholesky(mats2_o['s']))

#old version expects this under different key names
# mats2_o3['s_chol'] = s_chol_o3
# mats2_o3['s_oh'] = s_chol_o3
# mats2_o3['s_inv_oh'] = s_chol_o3.T

mats2_o['s_chol'] = s_chol_o
mats2_o['s_oh'] = s_chol_o
mats2_o['s_inv_oh'] = s_chol_o.T

#go through and make everything a list
# mats2_o3_l = {k: [mats2_o3[k]] for k in mats2_o3.keys()}
mats2_o_l = {k: [mats2_o[k]] for k in mats2_o.keys()}

#go through and additionally make everything a tensor
for k in mats2_o_l.keys():
    print(k)
#     o3k = mats2_o3_l[k]
    ok = mats2_o_l[k]
    if type(ok[0]) in [int, float]:
        print('integer/float from o3 dict')
#         mats2_o3_l[k] = [torch.Tensor(np.array(o3k))]
        mats2_o_l[k] = [torch.Tensor(np.array(ok))]
    elif not ok[0].shape:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0].reshape(1,))]
        mats2_o_l[k] = [torch.Tensor(ok[0].reshape(1,))]
    else:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0])]
        mats2_o_l[k] = [torch.Tensor(ok[0])]

dm_init
v
t
s
n_elec
n_atoms
e_nuc
mo_energy
mo_occ
e_ip
ip_idx
e_pretrained
eri
L
scaling
rho
ao_eval
grid_weights
fcenter
s_chol
s_oh
s_inv_oh


In [10]:
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA'
scf3 = lite_get_scf('MGGA', path=modelpath+'/xc')

sc=True
vvv=True

# res2_o3 = scf3([torch.Tensor(mats2_o3_l['dm_init'][0])], mats2_o3_l, sc=sc, verbose=vvv)
# res2_o3 = res2_o3['E']

res2_o = scf3([torch.Tensor(mats2_o_l['dm_init'][0])], mats2_o_l, sc=sc, verbose=vvv)
res2_o = res2_o['E']

# print('Dpyscf (Standalone Repo) Evaluated on:\nO3 = {}'.format(res2_o3))
print('Dpyscf (Standalone Repo + Softplus) Evaluated on:\nO = {}'.format(res2_o))

FREEC False
SCF Loop Beginning: 25 Steps
Step 0


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)
/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/ogSdpyscf/torch_routines.py:77: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2524.)
  e, c = torch.symeig(c

0 Energy: tensor([-74.8328], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-74.8646], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-74.8862], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-74.9006], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>), tensor([-74.9006], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-74.9110], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>), tensor([-74.9006], grad_fn=<AddBackward0>), tensor([-74.9110], grad_fn=<AddBackward0>

18 Energy: tensor([-74.9383], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>), tensor([-74.9006], grad_fn=<AddBackward0>), tensor([-74.9110], grad_fn=<AddBackward0>), tensor([-74.9187], grad_fn=<AddBackward0>), tensor([-74.9244], grad_fn=<AddBackward0>), tensor([-74.9284], grad_fn=<AddBackward0>), tensor([-74.9314], grad_fn=<AddBackward0>), tensor([-74.9335], grad_fn=<AddBackward0>), tensor([-74.9349], grad_fn=<AddBackward0>), tensor([-74.9360], grad_fn=<AddBackward0>), tensor([-74.9367], grad_fn=<AddBackward0>), tensor([-74.9373], grad_fn=<AddBackward0>), tensor([-74.9376], grad_fn=<AddBackward0>), tensor([-74.9379], grad_fn=<AddBackward0>), tensor([-74.9381], grad_fn=<AddBackward0>), tensor([-74.9382], grad_fn=<AddBackward0>), tensor([-74.9383], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-74.9384], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<

# Pasted Outputs

## Dpyscfl Output

In case the output gets erased for some reason and the above cell can't be run, the output is:
```
FREEC False
SCF Loop Beginning: 25 Steps
Step 0
Noise generation to avoid potential degeneracies
0 Energy: tensor([-225.1551], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-225.1555], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-225.1556], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>)]
Step 5
5 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 17

17 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 18
18 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>)]
SCF Loop Beginning: 25 Steps
Step 0

Noise generation to avoid potential degeneracies
0 Energy: tensor([-74.9761], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-74.9764], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-74.9764], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 5
5 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 17
17 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 18

18 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Dpyscf-lite (XCDiff) Evaluated on:
O3 = tensor([-225.1551, -225.1555, -225.1556, -225.1557, -225.1557, -225.1558,
        -225.1558, -225.1558, -225.1558, -225.1558, -225.1558, -225.1558,
        -225.1558, -225.1558, -225.1558, -225.1558, -225.1558, -225.1558,
        -225.1558, -225.1558, -225.1558, -225.1558, -225.1558, -225.1558,
        -225.1557], grad_fn=<CatBackward0>)
O = tensor([-74.9761, -74.9764, -74.9764, -74.9765, -74.9765, -74.9765, -74.9765,
        -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765,
        -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765,
        -74.9765, -74.9765, -74.9765, -74.9765], grad_fn=<CatBackward0>)

```

## Original Dpyscf Output

In case the output gets erased for some reason and the above cell can't be run, the output is:
```
FREEC False

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)

SCF Loop Beginning: 25 Steps
Step 0
0 Energy: tensor([-225.1550], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-118.4661], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-89.9705], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-76.8356], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-68.9395], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>)]
Step 5
5 Energy: tensor([-63.6487], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-60.0146], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-57.4984], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-55.7549], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-54.5512], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-53.7476], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-53.2192], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-52.8860], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-52.6544], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-52.5047], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-52.4044], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-52.3478], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>)]
Step 17
17 Energy: tensor([-52.3053], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>)]
Step 18

18 Energy: tensor([-52.2767], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-52.2593], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-52.2522], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-52.2416], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-52.2417], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>), tensor([-52.2417], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-52.2341], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>), tensor([-52.2417], grad_fn=<AddBackward0>), tensor([-52.2341], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-52.2365], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>), tensor([-52.2417], grad_fn=<AddBackward0>), tensor([-52.2341], grad_fn=<AddBackward0>), tensor([-52.2365], grad_fn=<AddBackward0>)]
SCF Loop Beginning: 25 Steps
Step 0
0 Energy: tensor([-74.9761], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-43.2015], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-34.2806], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-29.4604], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-26.3482], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>)]
Step 5

5 Energy: tensor([-24.1357], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-22.5100], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-21.3647], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-20.5265], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-19.8936], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-19.4032], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-19.0713], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-18.8011], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-18.6086], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-18.4598], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-18.3561], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-18.2716], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>)]
Step 17
17 Energy: tensor([-18.1856], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>)]
Step 18
18 Energy: tensor([-18.0978], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>)]
Step 19

19 Energy: tensor([-18.0366], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-18.0095], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-17.9769], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-17.9236], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>), tensor([-17.9236], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-17.8774], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>), tensor([-17.9236], grad_fn=<AddBackward0>), tensor([-17.8774], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-17.8375], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>), tensor([-17.9236], grad_fn=<AddBackward0>), tensor([-17.8774], grad_fn=<AddBackward0>), tensor([-17.8375], grad_fn=<AddBackward0>)]
Dpyscf (XCDiff) Evaluated on:
O3 = tensor([-225.1550, -118.4661,  -89.9705,  -76.8356,  -68.9395,  -63.6487,
         -60.0146,  -57.4984,  -55.7549,  -54.5512,  -53.7476,  -53.2192,
         -52.8860,  -52.6544,  -52.5047,  -52.4044,  -52.3478,  -52.3053,
         -52.2767,  -52.2593,  -52.2522,  -52.2416,  -52.2417,  -52.2341,
         -52.2365], grad_fn=<CatBackward0>)
O = tensor([-74.9761, -43.2015, -34.2806, -29.4604, -26.3482, -24.1357, -22.5100,
        -21.3647, -20.5265, -19.8936, -19.4032, -19.0713, -18.8011, -18.6086,
        -18.4598, -18.3561, -18.2716, -18.1856, -18.0978, -18.0366, -18.0095,
        -17.9769, -17.9236, -17.8774, -17.8375], grad_fn=<CatBackward0>)
```